In [13]:
##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
mainPath = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth'
outPath = r'P:\projects\RC10_ResultsSets'
summariesOutFolder = 'MineralsSummaries'
baseFolderEnd = 'BASE_RC10'
regContributorFileName = 'RegContributorDataGrid.csv'
outputCSVEnd = 'CatchmentToOutlet.csv'
baselineScenarioName = 'Baseline'
reportCardString = 'RC2019'

pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'


#altRegLinker = 'Catchmt'
standardRegName = 'RepReg'
#altRegColName = 'AUSGMCAS'
origLinker = 'ModelElement'

annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

#regionIDs = ['MW']
regionIDs = ['BM','BU','CY','FI','MW','WT']
Constituents = ['Sediment - Fine', 'N_DIN', 'P_FRP']
FUs = ['Sugarcane', 'Grazing Open', 'Grazing Forested', 'Bananas']

scenarioToFolderDict = {baselineScenarioName:baseFolderEnd}


In [14]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [15]:
if not os.path.exists(outPath + '\\' + summariesOutFolder):
    os.makedirs(outPath + '\\' + summariesOutFolder)
    print("Made dir: " + outPath + '\\' + summariesOutFolder)

In [16]:
def getPathInfo(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_' + scenarioToFolderDict[scenarioString] + '\\' + regContributorFileName
    fileOut = outPath + '\\' + summariesOutFolder + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut


In [17]:
def produceSummaryFromRegContrib(regionIDString, scenarioName):
    
    theFileIn, theFileOut = getPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    rawcontrib = pd.read_csv(theFileIn)
    
    #Filtering
    filtered = rawcontrib.loc[rawcontrib['Constituent'].isin(Constituents)]
    filtered2 = filtered.loc[filtered['FU'].isin(FUs)]
    
    filtered2['SCENARIO'] = scenarioName
    filtered2[annLoadToStream] = filtered2['LoadToStream (kg)'].div(filtered2['Num_Days']).mul(365.25)
    filtered2[annLoadToExport] = filtered2['LoadToRegExport (kg)'].div(filtered2['Num_Days']).mul(365.25)
    filtered2['AreaHA'] = filtered2['AreaM2'].div(10000)
    
#     if regionIDString in alternativeRegionsCSV:
#         print("Reading alternative regionalistaion for " + regionIDString)
#         altDetsDF = pd.read_csv(alternativeRegionsCSV[regionIDString])
#         combinedDF = pd.merge(rawcontrib, altDetsDF, how='left', left_on=[origLinker], right_on = [altRegLinker])
#         #Transfer vals
#         combinedDF['Rep_Region'] = combinedDF[altRegColName]
#         #Drop cols
#         #where 1 is the axis number (0 for rows and 1 for columns.)
#         combinedDF = combinedDF.drop(altRegColName, 1)
#         combinedDF = combinedDF.drop(standardRegName, 1)
#         combinedDF = combinedDF.drop(altRegLinker, 1)
#         rawcontrib = combinedDF
    
    #Ange wants catchment area, will need to grab it from the non-FU filtered bit...
    catchAllFUsArea = pd.DataFrame(filtered.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaM2':'first'})).reset_index()
    catchAllFUsArea['AreaHA'] = catchAllFUsArea['AreaM2'].div(10000)
    catchAreas = pd.DataFrame(catchAllFUsArea.groupby(['Rep_Region','ModelElement']).agg({'AreaHA':'sum'})).reset_index()
    catchAreas['CatchAreaHA'] = catchAreas['AreaHA']
    del catchAreas['AreaHA']
    
    catchFuAreas = pd.DataFrame(filtered2.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
    catchFuAreas['FUAreaHA'] = catchFuAreas['AreaHA']
    del catchFuAreas['AreaHA']
    #regFuArea = pd.DataFrame(catchFuArea.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
    #regLuseSummary = pd.DataFrame(rawcontrib[rawcontrib['Constituent'].isin(selectedConstituents)].groupby(['Rep_Region','Constituent','FU','Process']).agg({'AnnLoadToStreamKG':'sum','AnnLoadToExportKG':'sum'})).reset_index()
    regLuseSummary = pd.DataFrame(filtered2.groupby(['SCENARIO','Rep_Region','ModelElement','Constituent','FU']).agg({annLoadToStream:'sum',annLoadToExport:'sum'})).reset_index()
    
    ### Join/merge with Regional Areas
    regLuseSumPlusCatchArea = pd.merge(regLuseSummary, catchAreas, how='left', left_on=['Rep_Region','ModelElement'], right_on = ['Rep_Region','ModelElement'])
    regLuseSumPlusFUArea = pd.merge(regLuseSumPlusCatchArea, catchFuAreas, how='left', left_on=['Rep_Region','ModelElement','FU'], right_on = ['Rep_Region','ModelElement','FU'])
    
    regLuseSumPlusFUArea.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [18]:
#List out the processes

for theReg in regionIDs:
    
    for scenario in scenarioToFolderDict.keys():
        
        print("Doing: " + theReg + " " + scenario)
        produceSummaryFromRegContrib(theReg, scenario)



print("Finished first summary")

Doing: BM Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BM\Model_Outputs\BM_BASE_RC10\RegContributorDataGrid.csv Scenario: Baseline


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

Saved P:\projects\RC10_ResultsSets\MineralsSummaries\BM_RC2019_Baseline_CatchmentToOutlet.csv
Doing: BU Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_BASE_RC10\RegContributorDataGrid.csv Scenario: Baseline
Saved P:\projects\RC10_ResultsSets\MineralsSummaries\BU_RC2019_Baseline_CatchmentToOutlet.csv
Doing: CY Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\CY\Model_Outputs\CY_BASE_RC10\RegContributorDataGrid.csv Scenario: Baseline
Saved P:\projects\RC10_ResultsSets\MineralsSummaries\CY_RC2019_Baseline_CatchmentToOutlet.csv
Doing: FI Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\FI\Model_Outputs\FI_BASE_RC10\RegContributorDataGrid.csv Scenario: Baseline
Saved P:\projects\RC10_ResultsSets\MineralsSummaries\FI_RC2019_Baseline_CatchmentToOutlet.csv
Doing: MW Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\MW\M